### In this notebook a matrix was created for the LODES data for the affected areas

In [3]:
import zipfile 
import pysal as ps
%matplotlib inline
import numpy as np
import ast
import json
import urllib
import urllib2
from cStringIO import StringIO
import pandas as pd
import requests
import os
import geopandas as gpd
import matplotlib.pylab as pl
from fiona.crs import from_epsg
import matplotlib.pylab as pl
from matplotlib import colors
import shapely
import seaborn as sns
import gzip
import mapreduce as mr
import csv

In [4]:
fname = 'Data/CensusTracks.geojson'
tracks = gpd.GeoDataFrame.from_file(fname)
validCT = set(tracks[tracks.Within30min == True].boro_ct_2010)

In [5]:
def maptracks(row):
    if row['h_track'] in validCT:
        if ((row['h_track'][0]=='3')|(row['h_track'][0]=='4')):
            #print 'yes'
            if row['w_track'][0]=='1':
                #print 'yes'
                yield (row['w_track'], row['h_track'],int(row['S000']))

def reducehtracks(counts, pair_int):
    pair=pair_int[0:2]
    counts[pair] = counts.get(pair, 0)+pair_int[-1]
    return counts

with open('Data/True_nyod_JT00thegood_2014.csv', 'r') as fi:
    reader = csv.DictReader(fi)
    output2 = sorted(reduce(reducehtracks,list(mr.run(reader,maptracks)), {}).items())
    #
print len(output2)
output2[:10]

33609


[(('1000100', '3029700'), 2),
 (('1000100', '3030700'), 1),
 (('1000100', '3037700'), 1),
 (('1000100', '3038100'), 2),
 (('1000100', '3089400'), 1),
 (('1000100', '3090000'), 3),
 (('1000100', '3098200'), 1),
 (('1000100', '3098400'), 3),
 (('1000100', '3113000'), 2),
 (('1000100', '3114400'), 1)]

In [6]:
def maptracks2(row):
    yield row[0][0], row[0][1], row[1],


def reducehtracks(pair_int):
 
    return pair_int[0],pair_int[1],pair_int[2],

output3 = list(mr.run(output2,maptracks2))#,reducehtracks))
    #
print len(output3)
output3[:10]

33609


[('1000100', '3029700', 2),
 ('1000100', '3030700', 1),
 ('1000100', '3037700', 1),
 ('1000100', '3038100', 2),
 ('1000100', '3089400', 1),
 ('1000100', '3090000', 3),
 ('1000100', '3098200', 1),
 ('1000100', '3098400', 3),
 ('1000100', '3113000', 2),
 ('1000100', '3114400', 1)]

In [7]:
def maptracks2(both_int):
    yield both_int[0][0], both_int[0][1], both_int[1],


def reducehtracks(work_home_int):
    tupl= work_home_int[1],work_home_int[1],
 
    return work_home_int[0]

output4 = list(mr.run(output2,maptracks2,reducehtracks))
    #
print len(output4)
output4[:10]

285


['1000100',
 '1000201',
 '1000202',
 '1000500',
 '1000600',
 '1000700',
 '1000800',
 '1000900',
 '1001001',
 '1001002']

In [8]:
def maptracks2(both_int):
    yield both_int[0][0], both_int[0][1], both_int[1]


def reducehtracks3(work_home_int):
 
    return work_home_int[1],work_home_int[1]

output5 = list(mr.run(output2,maptracks2))
    #
print len(output5)
output5[:10]

33609


[('1000100', '3029700', 2),
 ('1000100', '3030700', 1),
 ('1000100', '3037700', 1),
 ('1000100', '3038100', 2),
 ('1000100', '3089400', 1),
 ('1000100', '3090000', 3),
 ('1000100', '3098200', 1),
 ('1000100', '3098400', 3),
 ('1000100', '3113000', 2),
 ('1000100', '3114400', 1)]

In [9]:
def maptracks2(w_h_int):
    yield w_h_int[1]
    
output6 = sorted(set(mr.run(output5,maptracks2)))
print len(output6)
output6[:10]

279


['3000100',
 '3002100',
 '3019100',
 '3024100',
 '3025300',
 '3025500',
 '3025700',
 '3025901',
 '3025902',
 '3026100']

In [10]:
homes=output6

In [11]:
jobs = output4

In [12]:
lodes = gpd.GeoDataFrame(columns=homes, index=jobs)

In [13]:
def maptodf(w_h_int):
    lodes.loc[w_h_int[0],w_h_int[1]]=w_h_int[2]
    
    return None

map(maptodf,output5)
print('done')

done


In [14]:
lodes.reset_index(inplace=True)
lodes.rename(inplace=True,columns={'index':'boro_ct_2010'})
lodes.head()


boro_ct_2010 3000100 3002100 3019100 3024100 3025300 3025500 3025700  \
0      1000100     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1      1000201       1     NaN     NaN     NaN       1     NaN     NaN   
2      1000202       1     NaN     NaN     NaN     NaN       1     NaN   
3      1000500     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4      1000600       2     NaN     NaN     NaN     NaN       1       1   

  3025901 3025902   ...   4059500 4059900 4060100 4060300 4061301 4061302  \
0     NaN     NaN   ...       NaN     NaN     NaN     NaN     NaN     NaN   
1     NaN     NaN   ...       NaN     NaN     NaN     NaN     NaN     NaN   
2     NaN     NaN   ...       NaN     NaN       2     NaN     NaN     NaN   
3     NaN     NaN   ...       NaN     NaN     NaN     NaN     NaN     NaN   
4     NaN     NaN   ...       NaN     NaN     NaN     NaN     NaN     NaN   

  4062300 4062500 4062700 4062900  
0     NaN     NaN     NaN     NaN  
1     NaN     NaN     NaN     NaN  
2     NaN     NaN     NaN     NaN  
3       1     NaN     NaN     NaN  
4     NaN     NaN     NaN     NaN  

[5 rows x 280 columns]

In [15]:
lodes.shape

(285, 280)

In [16]:
lodes.describe()

boro_ct_2010  3000100  3002100  3019100  3024100  3025300  3025500  \
count           285      146      146      133      118      129      139   
unique          285       32       40       22       21       21       23   
top         1016300        1        1        1        1        1        1   
freq              1       36       40       44       36       37       41   

        3025700  3025901  3025902   ...     4059500  4059900  4060100  \
count       118      107      121   ...         130       99      117   
unique       16       16       17   ...          22        9       14   
top           1        1        1   ...           1        1        1   
freq         47       41       50   ...          35       44       54   

        4060300  4061301  4061302  4062300  4062500  4062700  4062900  
count       103      142       41      100      106      119      111  
unique       13       24        4       12       13       16       16  
top           1        1        1        1        1        1        1  
freq         34       38       31       48       42       36       40  

[4 rows x 280 columns]

In [17]:
tracks.head(3)

Within15min  Within30min boro_code boro_ct_2010      boro_name cdeligibil  \
0            0            0         5      5000900  Staten Island          I   
1            0            0         5      5007400  Staten Island          I   
2            0            0         1      1009800      Manhattan          I   

  ct_2010 ctlabel                                           geometry id  \
0  000900       9  (POLYGON ((-74.07920577013245 40.6434307837456...  0   
1  007400      74  (POLYGON ((-74.05974734759452 40.5938486115672...  1   
2  009800      98  (POLYGON ((-73.96432543478758 40.7563815309909...  2   

  ntacode                                    ntaname  puma     shape_area  \
0    SI22  West New Brighton-New Brighton-St. George  3903  2497009.69813   
1    SI14            Grasmere-Arrochar-Ft. Wadsworth  3902  5788237.79601   
2    MN19                    Turtle Bay-East Midtown  3808  1906016.35002   

      shape_leng  
0  7729.01679376  
1  9902.94847281  
2  5534.19981063

In [18]:
tojoin = tracks[['geometry','boro_ct_2010']]
tojoin.head()

geometry boro_ct_2010
0  (POLYGON ((-74.07920577013245 40.6434307837456...      5000900
1  (POLYGON ((-74.05974734759452 40.5938486115672...      5007400
2  (POLYGON ((-73.96432543478758 40.7563815309909...      1009800
3  (POLYGON ((-73.96802436915851 40.7595781400528...      1010000
4  (POLYGON ((-73.97124277307127 40.7609364184790...      1010200

In [19]:
lodes = lodes.merge(tojoin, on='boro_ct_2010', how='left')

In [20]:
lodes.head(3)

boro_ct_2010 3000100 3002100 3019100 3024100 3025300 3025500 3025700  \
0      1000100     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1      1000201       1     NaN     NaN     NaN       1     NaN     NaN   
2      1000202       1     NaN     NaN     NaN     NaN       1     NaN   

  3025901 3025902                        ...                         4059900  \
0     NaN     NaN                        ...                             NaN   
1     NaN     NaN                        ...                             NaN   
2     NaN     NaN                        ...                             NaN   

  4060100 4060300 4061301 4061302 4062300 4062500 4062700 4062900  \
0     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
1     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
2       2     NaN     NaN     NaN     NaN     NaN     NaN     NaN   

                                            geometry  
0  (POLYGON ((-74.0438776163991 40.69018767537123...  
1  (POLYGON ((-73.98449647076764 40.7095096639394...  
2  (POLYGON ((-73.98237340572007 40.7055433504376...  

[3 rows x 281 columns]

In [32]:
lodes.to_file('Data/affectedjobs_homesmatrix')

with open('Data/fortopdestinations.geojson', 'w') as f:
    f.write(lodes.to_json())

In [33]:
lodesT = lodes.T
lodesT.head()

0        1        2        3        4        5        6    \
boro_ct_2010  1000100  1000201  1000202  1000500  1000600  1000700  1000800   
3000100           NaN        1        1      NaN        2       45        1   
3002100           NaN      NaN      NaN      NaN      NaN       45        1   
3019100           NaN      NaN      NaN      NaN      NaN       20        1   
3024100           NaN      NaN      NaN      NaN      NaN        8      NaN   

                  7        8        9     ...         275      276      277  \
boro_ct_2010  1000900  1001001  1001002   ...     1029300  1029500  1029700   
3000100            34      NaN      NaN   ...         NaN      NaN      NaN   
3002100            48      NaN      NaN   ...         NaN      NaN      NaN   
3019100            24      NaN      NaN   ...         NaN      NaN      NaN   
3024100            23      NaN      NaN   ...         NaN      NaN      NaN   

                  278      279      280      281      282      283      284  
boro_ct_2010  1029900  1030300  1030700  1030900  1031703  1031704  1031900  
3000100           NaN      NaN      NaN      NaN        8      NaN        2  
3002100           NaN      NaN      NaN      NaN        9      NaN        1  
3019100           NaN      NaN      NaN      NaN        3      NaN      NaN  
3024100           NaN      NaN      NaN      NaN      NaN      NaN      NaN  

[5 rows x 285 columns]

In [34]:
lodesT.columns = lodesT.iloc[0]

In [35]:
lodesT.head(3)

boro_ct_2010  1000100  1000201  1000202  1000500  1000600  1000700  1000800  \
boro_ct_2010  1000100  1000201  1000202  1000500  1000600  1000700  1000800   
3000100           NaN        1        1      NaN        2       45        1   
3002100           NaN      NaN      NaN      NaN      NaN       45        1   

boro_ct_2010  1000900  1001001  1001002   ...     1029300  1029500  1029700  \
boro_ct_2010  1000900  1001001  1001002   ...     1029300  1029500  1029700   
3000100            34      NaN      NaN   ...         NaN      NaN      NaN   
3002100            48      NaN      NaN   ...         NaN      NaN      NaN   

boro_ct_2010  1029900  1030300  1030700  1030900  1031703  1031704  1031900  
boro_ct_2010  1029900  1030300  1030700  1030900  1031703  1031704  1031900  
3000100           NaN      NaN      NaN      NaN        8      NaN        2  
3002100           NaN      NaN      NaN      NaN        9      NaN        1  

[3 rows x 285 columns]

In [36]:
lodesT = lodesT.iloc[1:-1,:]

In [37]:
lodesT.reset_index(inplace=True)
lodesT.rename(inplace=True,columns={'index':'boro_ct_2010'})
lodesT.head()

boro_ct_2010 boro_ct_2010 1000100 1000201 1000202 1000500 1000600 1000700  \
0                 3000100     NaN       1       1     NaN       2      45   
1                 3002100     NaN     NaN     NaN     NaN     NaN      45   
2                 3019100     NaN     NaN     NaN     NaN     NaN      20   
3                 3024100     NaN     NaN     NaN     NaN     NaN       8   
4                 3025300     NaN       1     NaN     NaN     NaN      13   

boro_ct_2010 1000800 1000900 1001001   ...   1029300 1029500 1029700 1029900  \
0                  1      34     NaN   ...       NaN     NaN     NaN     NaN   
1                  1      48     NaN   ...       NaN     NaN     NaN     NaN   
2                  1      24     NaN   ...       NaN     NaN     NaN     NaN   
3                NaN      23     NaN   ...       NaN     NaN     NaN     NaN   
4                  1      11     NaN   ...       NaN     NaN     NaN       2   

boro_ct_2010 1030300 1030700 1030900 1031703 1031704 1031900  
0                NaN     NaN     NaN       8     NaN       2  
1                NaN     NaN     NaN       9     NaN       1  
2                NaN     NaN     NaN       3     NaN     NaN  
3                NaN     NaN     NaN     NaN     NaN     NaN  
4                NaN     NaN     NaN       1       2     NaN  

[5 rows x 286 columns]

In [38]:
lodesT.index.rename('', inplace=True)

In [39]:
lodesT.head()

boro_ct_2010 boro_ct_2010 1000100 1000201 1000202 1000500 1000600 1000700  \
                                                                            
0                 3000100     NaN       1       1     NaN       2      45   
1                 3002100     NaN     NaN     NaN     NaN     NaN      45   
2                 3019100     NaN     NaN     NaN     NaN     NaN      20   
3                 3024100     NaN     NaN     NaN     NaN     NaN       8   
4                 3025300     NaN       1     NaN     NaN     NaN      13   

boro_ct_2010 1000800 1000900 1001001   ...   1029300 1029500 1029700 1029900  \
                                       ...                                     
0                  1      34     NaN   ...       NaN     NaN     NaN     NaN   
1                  1      48     NaN   ...       NaN     NaN     NaN     NaN   
2                  1      24     NaN   ...       NaN     NaN     NaN     NaN   
3                NaN      23     NaN   ...       NaN     NaN     NaN     NaN   
4                  1      11     NaN   ...       NaN     NaN     NaN       2   

boro_ct_2010 1030300 1030700 1030900 1031703 1031704 1031900  
                                                              
0                NaN     NaN     NaN       8     NaN       2  
1                NaN     NaN     NaN       9     NaN       1  
2                NaN     NaN     NaN       3     NaN     NaN  
3                NaN     NaN     NaN     NaN     NaN     NaN  
4                NaN     NaN     NaN       1       2     NaN  

[5 rows x 286 columns]

In [40]:
lodesT = lodesT.merge(tojoin, on='boro_ct_2010', how='left')

In [41]:
lodesT.head()

boro_ct_2010 boro_ct_2010 1000100 1000201 1000202 1000500 1000600 1000700  \
0                 3000100     NaN       1       1     NaN       2      45   
1                 3002100     NaN     NaN     NaN     NaN     NaN      45   
2                 3019100     NaN     NaN     NaN     NaN     NaN      20   
3                 3024100     NaN     NaN     NaN     NaN     NaN       8   
4                 3025300     NaN       1     NaN     NaN     NaN      13   

boro_ct_2010 1000800 1000900 1001001  \
0                  1      34     NaN   
1                  1      48     NaN   
2                  1      24     NaN   
3                NaN      23     NaN   
4                  1      11     NaN   

boro_ct_2010                        ...                         1029500  \
0                                   ...                             NaN   
1                                   ...                             NaN   
2                                   ...                             NaN   
3                                   ...                             NaN   
4                                   ...                             NaN   

boro_ct_2010 1029700 1029900 1030300 1030700 1030900 1031703 1031704 1031900  \
0                NaN     NaN     NaN     NaN     NaN       8     NaN       2   
1                NaN     NaN     NaN     NaN     NaN       9     NaN       1   
2                NaN     NaN     NaN     NaN     NaN       3     NaN     NaN   
3                NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN   
4                NaN       2     NaN     NaN     NaN       1       2     NaN   

boro_ct_2010                                           geometry  
0             (POLYGON ((-73.99093201741492 40.7007662160486...  
1             (POLYGON ((-73.97905767012279 40.7059528891005...  
2             (POLYGON ((-73.96015854658333 40.6941173091560...  
3             (POLYGON ((-73.95146465503193 40.6898633417958...  
4             (POLYGON ((-73.94891392355648 40.6939048667599...  

[5 rows x 287 columns]

In [42]:
lodesT.to_file('Data/affectedhomes_jobsmatrix')

In [43]:
with open('Data/fororigins.geojson', 'w') as f:
    f.write(lodesT.to_json())